## Question Answering System using End to End Memory Networks

## Imports

In [1]:
import keras
from keras.models import Sequential, Model
from keras.layers import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout
from keras.layers import add, dot, concatenate
from keras.layers import LSTM, GRU
from keras.utils.data_utils import get_file
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import backend as K

from functools import reduce
import tarfile
import numpy as np
import re

import IPython
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

### Helper functions

In [2]:
def tokenize(sent):
    if sent is not None:
        return [x.strip() for x in re.split('\W+', sent) if x is not None and x.strip()]
    return []

In [3]:
def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true, only the sentences
    that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data


In [4]:
def get_stories(f, only_supporting=False, max_length=None):
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data if not max_length or len(flatten(story)) < max_length]
    return data

In [5]:
def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    X = []
    Xq = []
    Y = []
    for story, query, answer in data:
        x = [word_idx[w] for w in story]
        xq = [word_idx[w] for w in query]
        # let's not forget that index 0 is reserved
        y = np.zeros(len(word_idx) + 1)
        y[word_idx[answer]] = 1
        X.append(x)
        Xq.append(xq)
        Y.append(y)
    return (pad_sequences(X, maxlen=story_maxlen),
            pad_sequences(Xq, maxlen=query_maxlen), np.array(Y))


In [6]:
class TrainingVisualizer(keras.callbacks.History):
    def on_epoch_end(self, epoch, logs={}):
        super().on_epoch_end(epoch, logs)
        IPython.display.clear_output(wait=True)
        pd.DataFrame({key: value for key, value in self.history.items() if key.endswith('loss')}).plot()
        axes = pd.DataFrame({key: value for key, value in self.history.items() if key.endswith('acc')}).plot()
        axes.set_ylim([0, 1])
        plt.show()

## Downloading the dataset

In [2]:
try:
    path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise
tar = tarfile.open(path)

## Getting train and test stories

In [8]:
challenges = {
    # QA1 with 10,000 samples
    'single_supporting_fact_10k': 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt',
    # QA2 with 10,000 samples
    'two_supporting_facts_10k': 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt',
}
challenge_type = 'single_supporting_fact_10k'
challenge = challenges[challenge_type]

print('Extracting stories for the challenge:', challenge_type)
train_stories = get_stories(tar.extractfile(challenge.format('train')))
test_stories = get_stories(tar.extractfile(challenge.format('test')))

Extracting stories for the challenge: single_supporting_fact_10k


In [9]:
len(train_stories), len(test_stories)

(10000, 1000)

## Preprocessing the data

In [10]:
vocab = set()
for story, q, answer in train_stories + test_stories:
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)

# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
story_maxlen = max(map(len, (x for x, _, _ in train_stories + test_stories)))
query_maxlen = max(map(len, (x for _, x, _ in train_stories + test_stories)))

In [11]:
story_maxlen, query_maxlen

(58, 3)

In [12]:
print('-')
print('Vocab size:', vocab_size, 'unique words')
print('Story max length:', story_maxlen, 'words')
print('Query max length:', query_maxlen, 'words')
print('Number of training stories:', len(train_stories))
print('Number of test stories:', len(test_stories))
print('-')
print('Here\'s what a "story" tuple looks like (input, query, answer):')
print(train_stories[0])
print('-')
print('Vectorizing the word sequences...')

word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
idx_word = dict((i+1, c) for i,c in enumerate(vocab))
inputs_train, queries_train, answers_train = vectorize_stories(train_stories,
                                                               word_idx,
                                                               story_maxlen,
                                                               query_maxlen)
inputs_test, queries_test, answers_test = vectorize_stories(test_stories,
                                                            word_idx,
                                                            story_maxlen,
                                                            query_maxlen)


-
Vocab size: 20 unique words
Story max length: 58 words
Query max length: 3 words
Number of training stories: 10000
Number of test stories: 1000
-
Here's what a "story" tuple looks like (input, query, answer):
(['Mary', 'moved', 'to', 'the', 'bathroom', 'John', 'went', 'to', 'the', 'hallway'], ['Where', 'is', 'Mary'], 'bathroom')
-
Vectorizing the word sequences...


In [13]:
inputs_train.shape, queries_train.shape, answers_train.shape

((10000, 58), (10000, 3), (10000, 20))

In [14]:
print('-')
print('inputs: integer tensor of shape (samples, max_length)')
print('inputs_train shape:', inputs_train.shape)
print('inputs_test shape:', inputs_test.shape)
print('-')
print('queries: integer tensor of shape (samples, max_length)')
print('queries_train shape:', queries_train.shape)
print('queries_test shape:', queries_test.shape)
print('-')
print('answers: binary (1 or 0) tensor of shape (samples, vocab_size)')
print('answers_train shape:', answers_train.shape)
print('answers_test shape:', answers_test.shape)
print('-')
print('Compiling...')


-
inputs: integer tensor of shape (samples, max_length)
inputs_train shape: (10000, 58)
inputs_test shape: (1000, 58)
-
queries: integer tensor of shape (samples, max_length)
queries_train shape: (10000, 3)
queries_test shape: (1000, 3)
-
answers: binary (1 or 0) tensor of shape (samples, vocab_size)
answers_train shape: (10000, 20)
answers_test shape: (1000, 20)
-
Compiling...


In [15]:
train_epochs = 100
batch_size = 32
lstm_size = 64

## Building the model

In [16]:
# placeholders
input_sequence = Input((story_maxlen,))
question = Input((query_maxlen,))

print('Input sequence:', input_sequence)
print('Question:', question)

# encoders
# embed the input sequence into a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,
                              output_dim=64))
input_encoder_m.add(Dropout(0.3))
# output: (samples, story_maxlen, embedding_dim)

# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,
                              output_dim=query_maxlen))
input_encoder_c.add(Dropout(0.3))
# output: (samples, story_maxlen, query_maxlen)

# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=query_maxlen))
question_encoder.add(Dropout(0.3))
# output: (samples, query_maxlen, embedding_dim)

# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
print('Input encoded m', input_encoded_m)
input_encoded_c = input_encoder_c(input_sequence)
print('Input encoded c', input_encoded_c)
question_encoded = question_encoder(question)
print('Question encoded', question_encoded)


# compute a 'match' between the first input vector sequence
# and the question vector sequence
# shape: `(samples, story_maxlen, query_maxlen)
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
print(match.shape)
match = Activation('softmax')(match)
print('Match shape', match)

# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)
print('Response shape', response)

# concatenate the response vector with the question vector sequence
answer = concatenate([response, question_encoded])
print('Answer shape', answer)

#answer = LSTM(lstm_size, return_sequences=True)(answer)  # Generate tensors of shape 32
#answer = Dropout(0.3)(answer)
answer = LSTM(lstm_size)(answer)  # Generate tensors of shape 32
answer = Dropout(0.3)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

Input sequence: KerasTensor(type_spec=TensorSpec(shape=(None, 58), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")
Question: KerasTensor(type_spec=TensorSpec(shape=(None, 3), dtype=tf.float32, name='input_2'), name='input_2', description="created by layer 'input_2'")
Input encoded m KerasTensor(type_spec=TensorSpec(shape=(None, 58, 64), dtype=tf.float32, name=None), name='sequential/dropout/Identity:0', description="created by layer 'sequential'")
Input encoded c KerasTensor(type_spec=TensorSpec(shape=(None, 58, 3), dtype=tf.float32, name=None), name='sequential_1/dropout_1/Identity:0', description="created by layer 'sequential_1'")
Question encoded KerasTensor(type_spec=TensorSpec(shape=(None, 3, 64), dtype=tf.float32, name=None), name='sequential_2/dropout_2/Identity:0', description="created by layer 'sequential_2'")
(None, 58, 3)
Match shape KerasTensor(type_spec=TensorSpec(shape=(None, 58, 3), dtype=tf.float32, name=None), name='activati

In [17]:
# build the final model
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

## Model summary

In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 58)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 3)]          0           []                               
                                                                                                  
 sequential (Sequential)        (None, None, 64)     1280        ['input_1[0][0]']                
                                                                                                  
 sequential_2 (Sequential)      (None, 3, 64)        1280        ['input_2[0][0]']                
                                                                                              

## Training the model and using Keras Callbacks for Visualization

In [19]:
model.fit([inputs_train, queries_train], answers_train, batch_size, train_epochs,
          validation_data=([inputs_test, queries_test], answers_test))

model.save('model.h5')

Epoch 1/100
313/313 [==============================] - 5s 9ms/step - loss: 1.8936 - accuracy: 0.1643 - val_loss: 1.7872 - val_accuracy: 0.1820
Epoch 2/100
313/313 [==============================] - 2s 7ms/step - loss: 1.7199 - accuracy: 0.2541 - val_loss: 1.5755 - val_accuracy: 0.3490
Epoch 3/100
313/313 [==============================] - 2s 7ms/step - loss: 1.5646 - accuracy: 0.3652 - val_loss: 1.5057 - val_accuracy: 0.3900
Epoch 4/100
313/313 [==============================] - 2s 6ms/step - loss: 1.5078 - accuracy: 0.4045 - val_loss: 1.4284 - val_accuracy: 0.4350
Epoch 5/100
313/313 [==============================] - 2s 7ms/step - loss: 1.4456 - accuracy: 0.4332 - val_loss: 1.3608 - val_accuracy: 0.4700
Epoch 6/100
313/313 [==============================] - 2s 7ms/step - loss: 1.4186 - accuracy: 0.4461 - val_loss: 1.3848 - val_accuracy: 0.4630
Epoch 7/100
313/313 [==============================] - 2s 7ms/step - loss: 1.3878 - accuracy: 0.4616 - val_loss: 1.3439 - val_accuracy: 0.4930

Epoch 58/100
313/313 [==============================] - 2s 7ms/step - loss: 0.2364 - accuracy: 0.9158 - val_loss: 0.2384 - val_accuracy: 0.9170
Epoch 59/100
313/313 [==============================] - 2s 7ms/step - loss: 0.2332 - accuracy: 0.9167 - val_loss: 0.2342 - val_accuracy: 0.9240
Epoch 60/100
313/313 [==============================] - 2s 8ms/step - loss: 0.2296 - accuracy: 0.9205 - val_loss: 0.2198 - val_accuracy: 0.9310
Epoch 61/100
313/313 [==============================] - 2s 7ms/step - loss: 0.2107 - accuracy: 0.9271 - val_loss: 0.2408 - val_accuracy: 0.9160
Epoch 62/100
313/313 [==============================] - 3s 8ms/step - loss: 0.2122 - accuracy: 0.9259 - val_loss: 0.2134 - val_accuracy: 0.9270
Epoch 63/100
313/313 [==============================] - 3s 8ms/step - loss: 0.1959 - accuracy: 0.9315 - val_loss: 0.2107 - val_accuracy: 0.9260
Epoch 64/100
313/313 [==============================] - 3s 8ms/step - loss: 0.1888 - accuracy: 0.9360 - val_loss: 0.2057 - val_accuracy:

## Test Result Analysis

In [20]:
for i in range(0,10):
        current_inp = test_stories[i]
        current_story, current_query, current_answer = vectorize_stories([current_inp], word_idx, story_maxlen, query_maxlen)
        current_prediction = model.predict([current_story, current_query])
        current_prediction = idx_word[np.argmax(current_prediction)]
        print(' '.join(current_inp[0]), ' '.join(current_inp[1]), '| Prediction:', current_prediction, '| Ground Truth:', current_inp[2])
        print("-----------------------------------------------------------------------------------------")


1/1 [==============================] - 1s 1s/step
John travelled to the hallway Mary journeyed to the bathroom Where is John | Prediction: hallway | Ground Truth: hallway
-----------------------------------------------------------------------------------------
1/1 [==============================] - 0s 31ms/step
John travelled to the hallway Mary journeyed to the bathroom Daniel went back to the bathroom John moved to the bedroom Where is Mary | Prediction: bathroom | Ground Truth: bathroom
-----------------------------------------------------------------------------------------
1/1 [==============================] - 0s 31ms/step
John travelled to the hallway Mary journeyed to the bathroom Daniel went back to the bathroom John moved to the bedroom John went to the hallway Sandra journeyed to the kitchen Where is Sandra | Prediction: kitchen | Ground Truth: kitchen
-----------------------------------------------------------------------------------------
1/1 [=============================